In [6]:
# processing
import operator
from operator import methodcaller
import csv
import re
import numpy as np
import pandas as pd
from pprint import pprint
import string
import math
import itertools

# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import HdpModel
from gensim.models import TfidfModel

# plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

# sci-kit
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import feature_extraction

In [ ]:
blacklist = [t.strip() for t in next(csv.reader(open("tools\\blacklist.csv", 'r')))]
levels = [1, 2, 3]

# format [term, orig, sentence, docID]
inPath = "raw.csv"

inFile = open(inPath, 'r')
inReader = csv.reader(inFile)

docTokens = dict()

# ignore headers
next(inReader)


for inRow in inReader:
    term = inRow[0]
    sentence = inRow[2]
    docID = inRow[3]
    
    # find acceptable tokens only
    token = "_".join([t for t in term.split(":") if re.match(r'[^\W\d]*$', t) and not t in blacklist])
    
    # calculate new term level
    level = token.count("_")
    
    # if acceptable, add to dictionary
    if level in levels and not token in blacklist and len(token) > 0:
        if docID in docTokens:
            docTokens[docID] += [token]
        else:
            docTokens[docID] = [token]
                        
docIDs = list(docTokens.keys())
texts = list(docTokens.values())


In [18]:
dictionary = corpora.Dictionary(texts)
print(len(dictionary))

dictionary.filter_extremes(no_below=5, no_above=0.25, keep_n=15000)
print(len(dictionary))

corpus = [dictionary.doc2bow(text) for text in texts]


tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

16391
396


In [19]:
topics = 8
passes = 10
iterations = 500

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=topics, id2word=dictionary, passes=passes, workers =4)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic) + "\n")


Topic: 0 Word: 0.030*"c_h" + 0.026*"o_interaction" + 0.023*"water_molecule" + 0.023*"metal_atom" + 0.022*"bond_length" + 0.021*"crystal_structure" + 0.018*"benzene_ring" + 0.018*"o_atom" + 0.015*"intermolecular_n_h" + 0.015*"dihedral_angle"

Topic: 1 Word: 0.028*"intermolecular_interaction" + 0.027*"n_hydrogen_bond" + 0.025*"dihedral_angle" + 0.024*"square_planar" + 0.021*"coordination_geometry" + 0.021*"solvent_molecule" + 0.020*"n_hydrogen" + 0.020*"phenyl_ring" + 0.019*"n_h" + 0.018*"weak_c_h"

Topic: 2 Word: 0.031*"room_temperature" + 0.023*"crystal_structure" + 0.021*"x_ray" + 0.020*"active_site" + 0.018*"octahedral_coordination" + 0.017*"x_ray_scatter" + 0.017*"solid_state" + 0.015*"water_molecule" + 0.015*"space_group" + 0.014*"diffraction_datum"

Topic: 3 Word: 0.046*"aring_resolution" + 0.043*"unit_cell" + 0.039*"space_group" + 0.031*"unit_cell_parameter" + 0.027*"single_crystal" + 0.026*"asymmetric_unit" + 0.024*"crystal_structure" + 0.023*"c_terminal" + 0.020*"molecular_repl